In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk
from tensorflow.keras import layers, models
from tensorflow.keras.layers import  Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [2]:
def preprocess_image(image):
    if image.ndim == 2:  # 如果图像已经是灰度图像
        gray = image
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (224, 224))
    normalized = resized / 255.0
    return normalized

In [42]:
def load_lfw_dataset(data_path, image_size=(224,224)):
    images = []
    labels = []
    unique_labels = {}

    for person in os.listdir(data_path):
        person_dir = os.path.join(data_path, person)
        if os.path.isdir(person_dir):
            for image_file in os.listdir(person_dir):
                image_path = os.path.join(person_dir, image_file)
                image = cv2.imread(image_path)
                image = preprocess_image(image)
                image = cv2.resize(image, image_size)
                images.append(image)

                # 分配一个数字标签给每个人物
                if person not in unique_labels:
                    unique_labels[person] = len(unique_labels)
                labels.append(unique_labels[person])

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

lfw_data_path = './data/face'
images, labels = load_lfw_dataset(lfw_data_path)
images = images[..., np.newaxis]  # 将图像维度从 (H, W) 扩展为 (H, W, 1)

# 划分训练集和测试集
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.25, random_state=42)



In [48]:
def create_model(num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [52]:
num_classes=len(np.unique(labels))
model=create_model(num_classes)


In [53]:


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

batch_size = 64
epochs = 30
model.fit(train_images, train_labels, steps_per_epoch=len(train_images) // batch_size, epochs=epochs, validation_data=(val_images,  val_labels))

Epoch 1/30
69/69 [==============================] - 13s 181ms/step - loss: 3.6554 - accuracy: 0.0436 - val_loss: 3.6109 - val_accuracy: 0.0574
Epoch 2/30
69/69 [==============================] - 13s 188ms/step - loss: 3.4761 - accuracy: 0.0749 - val_loss: 3.3293 - val_accuracy: 0.1032
Epoch 3/30
69/69 [==============================] - 12s 173ms/step - loss: 3.2154 - accuracy: 0.1251 - val_loss: 3.0595 - val_accuracy: 0.1511
Epoch 4/30
69/69 [==============================] - 12s 174ms/step - loss: 3.0053 - accuracy: 0.1639 - val_loss: 2.8184 - val_accuracy: 0.2058
Epoch 5/30
69/69 [==============================] - 12s 173ms/step - loss: 2.8199 - accuracy: 0.2063 - val_loss: 2.4925 - val_accuracy: 0.2713
Epoch 6/30
69/69 [==============================] - 12s 173ms/step - loss: 2.7128 - accuracy: 0.2296 - val_loss: 2.4488 - val_accuracy: 0.2895
Epoch 7/30
69/69 [==============================] - 12s 172ms/step - loss: 2.5303 - accuracy: 0.2773 - val_loss: 2.1947 - val_accuracy: 0.3576

In [7]:
model.save('face_recognition_model.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
loaded_model = tf.keras.models.load_model('face_recognition_model.h5')
def recognize_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(20, 20))
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = preprocess_image(face_roi)
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_roi = np.expand_dims(face_roi, axis=0)

        predictions = loaded_model.predict(face_roi)
        predicted_label = np.argmax(predictions)

        # 在原始图像上绘制检测到的人脸矩形和识别结果
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, f"Person {predicted_label}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    return image


In [8]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 173056)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               2

In [39]:
from PIL import Image
def load_image(label):
    global image_path
    image_path = filedialog.askopenfilename()
    image = Image.open(image_path)
    image.thumbnail((500, 500))
    image = ImageTk.PhotoImage(image)
    label.config(image=image)
    label.image = image

def process_image(label):
    image = cv2.imread(image_path)
    output_image = recognize_faces(image)
    output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
    output_image = Image.fromarray(output_image)
    output_image.thumbnail((500, 500))
    output_image = ImageTk.PhotoImage(output_image)
    label.config(image=output_image)
    label.image = output_image

In [40]:


# 创建Tkinter窗口
root = Tk()
root.title("Face Recognition")

# 添加画布和标签显示图像
canvas = Canvas(root)
canvas.pack()
label = Label(root)
label.pack()

button=Button(root,text="Open Image", command=lambda:load_image(label))
button.pack()

# 添加识别按钮
button = Button(root, text="Recognize Faces", command=lambda:process_image(label))
button.pack()

# 运行Tkinter主循环
root.mainloop()

[[440 144  87  87]]
(1, 40)
